search data obtained form 
https://digitallibrary.un.org/search?ln=en&as=1&rm=&sf=year&so=d&rg=100&c=United+Nations+Digital+Library+System&of=hb&fti=0&fct__1=Speeches&fti=0&as_query=JTdCJTIyZGF0ZV9zZWxlY3RvciUyMiUzQSU3QiUyMmRhdGVUeXBlJTIyJTNBJTIyY3JlYXRpb25fZGF0ZSUyMiUyQyUyMmRhdGVQZXJpb2QlMjIlM0ElMjJhbGx5ZWFycyUyMiUyQyUyMmRhdGVGcm9tJTIyJTNBJTIyJTIyJTJDJTIyZGF0ZVRvJTIyJTNBJTIyJTIyJTdEJTJDJTIyY2xhdXNlcyUyMiUzQSU1QiU3QiUyMnNlYXJjaEluJTIyJTNBJTIyYXV0aG9yJTIyJTJDJTIyY29udGFpbiUyMiUzQSUyMmV4YWN0LW1hdGNoJTIyJTJDJTIydGVybSUyMiUzQSUyMlVOLiUyMFNlY3JldGFyeS1HZW5lcmFsJTIyJTJDJTIyb3BlcmF0b3IlMjIlM0ElMjJBTkQlMjIlN0QlNUQlN0Q%3D&action_search=placeholder#searchresultsbox

In [189]:
import json
import xmltodict
from bs4 import BeautifulSoup  
import pandas as pd
import xml.etree.ElementTree as ET
import unicodedata
from datetime import date
import numpy as np  
import re
import logging


In [190]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

### load data

In [191]:
tree = ET.parse('data/all_SG_speeches_search.xml')
root = tree.getroot()

In [192]:
root.tag
namespace = {'nmsp': 'http://www.loc.gov/MARC21/slim'}

In [193]:
len(list(root.findall("nmsp:record", namespaces=namespace)))

1212

### transfrom into dataframe

In [194]:
data = []
for record in root.findall('nmsp:record', namespace):
    record_id = record.find("nmsp:controlfield[@tag='001']", namespace)
    field = record.find("nmsp:datafield[@tag='700']", namespace)
    speaker = field.find("nmsp:subfield[@code='a']", namespace)
    field = record.find("nmsp:datafield[@tag='710']", namespace)
    speaker_organization = field.find("nmsp:subfield[@code='a']", namespace)
    # field = record.find("nmsp:datafield[@tag='089']", namespace)
    # doc_type = field.find("nmsp:subfield[@code='a']", namespace)
    field = record.find("nmsp:datafield[@tag='791']", namespace)
    speech_code = field.find("nmsp:subfield[@code='a']", namespace)
    speech_code_searchable = field.find("nmsp:subfield[@code='q']", namespace)
    field = record.find("nmsp:datafield[@tag='992']", namespace)
    date = field.find("nmsp:subfield[@code='a']", namespace)
    data.append([record_id.text, 
                 speaker.text, 
                 speaker_organization.text, 
                #  doc_type.text, 
                 speech_code.text, 
                 speech_code_searchable.text if speech_code_searchable is not None else None, 
                 date.text])

In [195]:
records = pd.DataFrame(data, columns=['record_id', 'speaker', 'speaker_organization', 'speech_code', 'speech_code_searchable', 'date'])
records_len = len(records)
records

,record_id,speaker,speaker_organization,speech_code,speech_code_searchable,date
0,4090177,"Guterres, António, 1949-",UN. Secretary-General,S/PV.9988,None,2025-08-28
1,4087505,"Guterres, António, 1949-",UN. Secretary-General,S/PV.9962,None,2025-07-22
2,4085871,"Guterres, António, 1949-",UN. Secretary-General,S/PV.9941,SPV9941,2025-06-22
3,4084965,"Guterres, António, 1949-",UN. Secretary-General,S/PV.9939,SPV9939,2025-06-20
4,4084667,"Guterres, António, 1949-",UN. Secretary-General,S/PV.9938,SPV9938,2025-06-19
...,...,...,...,...,...,...
1207,3983935,"Lie, Trygve, 1896-1968",UN. Secretary-General,"E/SR.15[1946, 3rd sess.]",None,1946-10-01
1208,3983784,"Lie, Trygve, 1896-1968",UN. Secretary-General,"E/SR.14[1946, 3rd sess.]",None,1946-09-30
1209,3983385,"Lie, Trygve, 1896-1968",UN. Secretary-General,"E/SR.3[1946, 3rd sess.]",None,1946-09-12
1210,3985925,"Lie, Trygve, 1896-1968",UN. Secretary-General,"E/SR.5[1946, 2nd sess.]",None,1946-05-31


In [196]:
# for row in records.itertuples():
#     if '[' in row.speech_code:
#         log.info(row.speech_code)

### Fill up the missing speech code searchables

In [197]:
log.info(f"there are {len(records[records.speech_code_searchable.isna()])} missing search codes")

INFO:__main__:there are 72 missing search codes


In [198]:
renaming_list = [
    # the old E series under Trygve Lie is grouped into documents and therfore named a bit differently
    {'old_code': 'E/SR.1[1946, 2nd sess.]', 'new_code': 'E/SR.1-2'},
    {'old_code': 'E/SR.3[1946, 3rd sess.]', 'new_code': 'E/SR.3-4'},
    {'old_code': 'E/SR.5[1946, 2nd sess.]', 'new_code': 'E/SR.5-6'},
    {'old_code': 'E/SR.14[1946, 3rd sess.]', 'new_code': 'E/SR.14'},
    {'old_code': 'E/SR.15[1946, 3rd sess.]', 'new_code': 'E/SR.15-16'},
    {'old_code': 'E/SR.17[1946, 3rd sess.]', 'new_code': 'E/SR.17-18'},
    {'old_code': 'E/SR.19[1946, 3rd sess.]', 'new_code': 'E/SR.19-20'},
    {'old_code': 'E/SR.20[1946, 3rd sess.', 'remove': True}, # 19-20 is twice
    {'old_code': 'E/SR.21[1946, 3rd sess.]', 'new_code': 'E/SR.21'},
    
    # missing spaces
    {'old_code': 'A/78/PV.50(Resumption1)', 'new_code': 'A/78/PV.50 (Resumption 1)'},

    # there are some for which the corrections don't exist
    {'old_code': 'S/PV.2179andCorr.1', 'new_code': 'S/PV.2179'},
    {'old_code': 'S/PV.2051andCorr.1', 'new_code': 'S/PV.2051'},
    {'old_code': 'S/PV.1782andCorr.1', 'new_code': 'S/PV.1782'},

    # and the non existent ones
    # {'old_code': 'E/1978/SR.17', 'remove': True},
]

In [199]:
for ren in renaming_list:
    if 'new_code' in ren:
        records.speech_code = records.speech_code.replace(ren['old_code'], ren['new_code'])
        log.info(f"renamed {ren['old_code']} into {ren['new_code']}")
    elif 'remove' in ren:
        records = records[records.speech_code != ren['old_code']].copy()
        log.info(f"dropped {ren['old_code']}")
    else:
        log.warning(f"no action performed on {ren['old_code']}")
records.reset_index(inplace=True, drop=True)

INFO:__main__:renamed E/SR.1[1946, 2nd sess.] into E/SR.1-2
INFO:__main__:renamed E/SR.3[1946, 3rd sess.] into E/SR.3-4
INFO:__main__:renamed E/SR.5[1946, 2nd sess.] into E/SR.5-6
INFO:__main__:renamed E/SR.14[1946, 3rd sess.] into E/SR.14
INFO:__main__:renamed E/SR.15[1946, 3rd sess.] into E/SR.15-16
INFO:__main__:renamed E/SR.17[1946, 3rd sess.] into E/SR.17-18
INFO:__main__:renamed E/SR.19[1946, 3rd sess.] into E/SR.19-20
INFO:__main__:dropped E/SR.20[1946, 3rd sess.
INFO:__main__:renamed E/SR.21[1946, 3rd sess.] into E/SR.21
INFO:__main__:renamed A/78/PV.50(Resumption1) into A/78/PV.50 (Resumption 1)
INFO:__main__:renamed S/PV.2179andCorr.1 into S/PV.2179
INFO:__main__:renamed S/PV.2051andCorr.1 into S/PV.2051
INFO:__main__:renamed S/PV.1782andCorr.1 into S/PV.1782


now after naming them properly we can copy them over

In [200]:
records.speech_code_searchable = records.speech_code_searchable.fillna(records.speech_code)

### transfrom specific columns

In [201]:
# unicode normalize because they use differnt o accents for Guterres António
records.speaker = [unicodedata.normalize('NFKD',s) for s in records.speaker]
# date to date object
records.date = pd.to_datetime(records.date)

In [202]:
records.dtypes

record_id                         object
speaker                           object
speaker_organization              object
speech_code                       object
speech_code_searchable            object
date                      datetime64[ns]
dtype: object

In [203]:
records.speaker.unique()

array(['Guterres, António, 1949-', 'Ban, Ki-moon, 1944-',
       'Annan, Kofi, 1938-2018', 'Boutros-Ghali, Boutros, 1922-2016',
       'Pérez de Cuéllar, Javier, 1920-2020',
       'Waldheim, Kurt, 1918-2007', 'Thant, U, 1909-1974',
       'Hammarskjöld, Dag, 1905-1961', 'Lie, Trygve, 1896-1968'],
      dtype=object)

### remove speeches that are held out of term time

In [204]:
# all secretary general and their term dates
still_active_placeholder = '9999-12-31'
all_sgs = {
    'Guterres, António' : {
        'start': '2017-01-01',
        'end': still_active_placeholder
    },
    'Ban, Ki-moon' : {
        'start': '2007-01-01',
        'end': '2016-12-31',
    },
    'Annan, Kofi' : {
        'start': '1997-01-01',
        'end': '2006-12-31',
    },
    'Boutros-Ghali, Boutros' : {
        'start': '1992-01-01',
        'end': '1996-12-31',
    },
    'Pérez de Cuéllar, Javier' : {
        'start': '1982-01-01',
        'end': '1991-12-31',
    },
    'Waldheim, Kurt' : {
        'start': '1972-01-01',
        'end': '1981-12-31',
    },
    'Thant, U' : {
        'start': '1961-11-03',
        'end': '1971-12-31',
    },
    'Hammarskjöld, Dag' : {
        'start': '1953-04-10',
        'end': '1961-09-18',
    },
    'Lie, Trygve' : {
        'start': '1946-02-02',
        'end': '1952-11-10',
    }    
}
# all_sgs = json.load(json.dumps(all_sgs)

In [205]:
indices_to_drop = np.array([], dtype=int)
for sg in all_sgs:
    start_date = np.datetime64(all_sgs[sg]['start'])
    end_date = np.datetime64(all_sgs[sg]['end'])
    sg_records = records[records.speaker.str.contains(sg)] # get all rows for the SG
    new_indices = sg_records[sg_records.date < start_date].index #  get all rows with speeches not during his term
    if any(new_indices):
        indices_to_drop = np.concatenate((indices_to_drop, np.array(new_indices)))
log.info('will drop: ')
display(records.iloc[indices_to_drop])
records.drop(indices_to_drop, inplace=True)

log.info(f'{records_len - len(records)} indices were dropped because they were speeches out of term times')
records_len = len(records)
records.reset_index(drop=True, inplace=True)

INFO:__main__:will drop: 


,record_id,speaker,speaker_organization,speech_code,speech_code_searchable,date
537,592461,"Ban, Ki-moon, 1944-",UN. Secretary-General,A/61/PV.78,A61PV78,2006-12-14
538,588393,"Ban, Ki-moon, 1944-",UN. Secretary-General,A/61/PV.31,A61PV31,2006-10-13
798,388390,"Annan, Kofi, 1938-2018",UN. Secretary-General,A/C.3/51/SR.30,AC351SR30,1996-11-08
841,353838,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/47/PV.83,A47PV83,1991-12-10
843,346704,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/46/PV.59,A46PV59,1991-12-03


INFO:__main__:5 indices were dropped because they were speeches out of term times


### fill up search data - obsolete we take the given now

In [206]:
# def make_searchable(code):
#     code = code.replace('/', '\\/')
#     # code = code.replace('.', '')
#     code = code.replace('[', ' \\[')
#     code = code.replace(']', '\\]')
#     return code

# new_searchable = []
# for index, row in records.iterrows():
#     # new_sbl = row.speech_code_searchable if pd.notna(row.speech_code_searchable) else make_searchable(row.speech_code)
#     new_sbl = make_searchable(row.speech_code)
#     new_searchable.append(new_sbl)

# records.speech_code_searchable = new_searchable

### introducing some more meta data for easier saving later

add splitted date

In [207]:
records.date = pd.to_datetime(records.date)
records['year'] = records.date.dt.year
records['month'] = records.date.dt.month
records['day'] = records.date.dt.day

add splitted document name in body, sub body, document type and additional part

In [208]:
def document_code_breakup(document_code):
    document_code = str(document_code)
    body, sub_body, doc_number, doc_type, add_part = None, None, None, None, None
    splits = document_code.split('/')

    body = splits[0]

    back = re.match(r'(\w{1,4}\.\d{1,4})(.*)', splits[-1])
    doc_type, add_part = back.groups()
    if add_part:
        add_part = add_part.replace('(', '')
        add_part = add_part.replace(')', '')
        add_part = add_part.replace('[', '')
        add_part = add_part.replace(']', '')
        add_part = add_part.replace(' ', '_')
        add_part = add_part.replace(',', '')
        add_part = add_part.upper()
    else:
        add_part = None

    if len(splits) == 4:
        sub_body = splits[1]
        doc_number = splits[2]

    if len(splits) == 3:
        split = splits[1]
        is_year = re.fullmatch(r'\d{4}', split)
        is_session_nr = re.fullmatch(r'\d{2}', split)
        is_known_sub_body = re.fullmatch(r'(C\.\d{1,2})|(CN\.\d{1,2})|(S-\d{1,2})|(ES-\d{1,2})', split)
        if(is_year or is_session_nr):
            doc_number = split
        elif(is_known_sub_body):
            sub_body = split
        else:
            log.warning(f"unknown middle sub-body or doc-number in {split} for document code: {document_code}")

    return body, sub_body, doc_number, doc_type, add_part

In [209]:
bodies, sub_bodies, doc_numbers, doc_types, add_parts = [],[],[],[],[]
for ln in records.itertuples():
    # log.info(document_code_breakup(ln.speech_code))
    body, sub_body, doc_number, doc_type, add_part = document_code_breakup(ln.speech_code)
    bodies.append(body)
    sub_bodies.append(sub_body)
    doc_numbers.append(doc_number)
    doc_types.append(doc_type)
    add_parts.append(add_part)
records['body'] = bodies
records['sub_body'] = sub_bodies
records['doc_number'] = doc_numbers
records['doc_type'] = doc_types
records['add_part'] = add_parts

add file name

In [210]:
def build_file_name(metadata_row):
    # file_name = "UNSG_"
    file_name = str(metadata_row.body) + '_'
    file_name += str(metadata_row.year) + '_'
    # file_name += (metadata_row.body + '_') if metadata_row.body else ''
    file_name += (metadata_row.sub_body + '_') if metadata_row.sub_body else ''
    file_name += (metadata_row.doc_number + '_') if metadata_row.doc_number else ''
    file_name += (metadata_row.doc_type + '_') if metadata_row.doc_type else ''
    file_name += (metadata_row.add_part + '_') if metadata_row.add_part else ''
    # file_name += 'speeches.pdf'
    return file_name

In [211]:
records['file_prefix'] = records.apply(build_file_name, axis=1)
records['file_name_pdf'] = records.file_prefix + 'speeches.pdf'

In [212]:
records.rename(columns={'speech_code': 'document_symbol', 'speech_code_searchable': 'document_symbol_searchable'}, inplace=True)

In [213]:
log.info(f"records has a length of {len(records)}")
records.head(1)

INFO:__main__:records has a length of 1207


,record_id,speaker,speaker_organization,document_symbol,document_symbol_searchable,date,year,month,day,body,sub_body,doc_number,doc_type,add_part,file_prefix,file_name_pdf
0,4090177,"Guterres, António, 1949-",UN. Secretary-General,S/PV.9988,S/PV.9988,2025-08-28,2025,8,28,S,None,None,PV.9988,None,S_2025_PV.9988_,S_2025_PV.9988_speeches.pdf


## save

In [214]:
records.to_csv('data/speech_records.csv', index=False)